In [1]:
import os
import urllib.request

base_url = "https://physionet.org/files/eegmmidb/1.0.0/"
output_dir = "../data/raw/physionet"
os.makedirs(output_dir, exist_ok=True)

subjects = [f"S{str(i).zfill(3)}" for i in range(1, 11)]  # S001 à S010
runs = [f"R{str(i).zfill(2)}" for i in range(1, 15)]      # R01 à R14

downloaded = 0
for subject in subjects:
    for run in runs:
        filename = f"{subject}{run}.edf"
        url = f"{base_url}{subject}/{filename}"
        local_path = os.path.join(output_dir, filename)

        if os.path.exists(local_path):
            continue  # déjà présent

        try:
            print(f"Téléchargement de {filename}...")
            urllib.request.urlretrieve(url, local_path)
            downloaded += 1
        except Exception as e:
            print(f"❌ Échec : {filename} ({e})")

print(f"\n✅ Téléchargements terminés : {downloaded} fichiers")



✅ Téléchargements terminés : 0 fichiers


In [2]:
import os
import glob
import mne
import numpy as np

# 📁 Chemin absolu vers le dossier contenant les fichiers EDF
data_dir = r"C:/Users/Antoi/Documents/EEG-BCI/data/raw/physionet"

# 🔍 Recherche de tous les fichiers EDF dans le dossier
edf_files = sorted(glob.glob(os.path.join(data_dir, "*.edf")))
if len(edf_files) == 0:
    raise FileNotFoundError(f"Aucun fichier EDF trouvé dans {data_dir}")

X_all = []
y_all = []

for file in edf_files:
    print(f"\n📄 Traitement du fichier : {os.path.basename(file)}")

    try:
        # 📥 Chargement des données EEG
        raw = mne.io.read_raw_edf(file, preload=True)
        raw.set_eeg_reference('average', projection=True)

        # 🧠 Appliquer un montage standard
        montage = mne.channels.make_standard_montage("standard_1020")
        raw.set_montage(montage, on_missing="ignore")

        # ❌ Supprimer les canaux problématiques pour la topomap
        bad_chs = ['FCZ', 'CZ', 'CPZ', 'FP1', 'FPZ', 'FP2',
                   'AFZ', 'FZ', 'PZ', 'POZ', 'OZ', 'IZ']
        raw.pick_channels([ch for ch in raw.ch_names if ch not in bad_chs])

        # 🎚️ Filtrage passe-bande 1–40 Hz
        raw.filter(1., 40., fir_design='firwin')

        # 🏷️ Extraction des événements via annotations
        events, _ = mne.events_from_annotations(raw)
        event_id = dict(rest=1, left=2, right=3)

        # 📦 Extraction des epochs
        epochs = mne.Epochs(raw, events, event_id=event_id,
                            tmin=0, tmax=2, baseline=None,
                            preload=True, detrend=1)
        
        # ❗ Skip si aucune epoch détectée
        if len(epochs) == 0:
            print(f"⚠️ Aucune epoch valide dans {file}, ignoré.")
            continue

        # 🔢 Récupération des données [N, C, T]
        data = epochs.get_data()
        labels = epochs.events[:, 2]

        # ⚙️ Normalisation (z-score par trial)
        data = (data - data.mean(axis=-1, keepdims=True)) / data.std(axis=-1, keepdims=True)

        # 🧩 Ajout aux données globales
        X_all.append(data)
        y_all.append(labels)

    except Exception as e:
        print(f"❌ Erreur sur {file} : {e}")

# 🔄 Concaténation finale
X = np.concatenate(X_all, axis=0)
y = np.concatenate(y_all, axis=0)

# 💾 Sauvegarde des arrays
np.save("X_windows.npy", X)
np.save("y_windows.npy", y)

print(f"\n✅ Données prêtes pour DeepConvNet : X = {X.shape}, y = {y.shape}")



📄 Traitement du fichier : S001R01.edf
Extracting EDF parameters from C:\Users\Antoi\Documents\EEG-BCI\data\raw\physionet\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Up

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'),

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


0 bad epochs dropped

📄 Traitement du fichier : S001R04.edf
Extracting EDF parameters from C:\Users\Antoi\Documents\EEG-BCI\data\raw\physionet\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff fr

...

Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'),

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


Reading 0 ... 19679  =      0.000 ...   122.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished



✅ Données prêtes pour DeepConvNet : X = (3600, 64, 321), y = (3600,)


In [3]:
from collections import Counter
print(Counter(y))


Counter({np.int64(1): 1800, np.int64(2): 903, np.int64(3): 897})
